In [2]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import pandas as pd
import math
 
server = SSHTunnelForwarder(
    ssh_address_or_host=('10.12.0.8', 2043),
    ssh_username='root', 
    ssh_password = 'ynby20250625', 
 
    
    remote_bind_address=('localhost' , 5432)) 
server.start()
conn = psycopg2.connect(database = 'FAERS20-24', 
                            user = 'postgres',     
                            password = 'woshipostgres', 
                            host = '127.0.0.1',   
                            port = server.local_bind_port)

In [3]:
def get_drug_reaction(drugid, con = conn):
    sql = (f"SELECT DISTINCT reactid FROM cdir_fo WHERE drugbank_id = '{drugid}';")
    drug_reaction = pd.read_sql(sql, con = conn)
    #print(drug_reaction)
    reactids = drug_reaction.iloc[:, 0]
    return reactids

def count_drug(drugid, con = conn):
    '''
    Figure out the value 'a+c'(the sum of recordings of all reactions 
    triggered by the given drug) in the Disproportional Method
    '''
    sql = (f"SELECT count FROM drugcount WHERE drugbank_id = '{drugid}';")
    drugcount = pd.read_sql(sql, con=conn)
    return drugcount['count'].values[0]

def count_react(reactid, con = conn):
    '''
    Figure out the value 'a+b'(the sum of recordings of the given reaction 
    ) in the Disproportional Method
    '''
    sql = (f"SELECT count FROM reactcount WHERE reactid = '{reactid}';")
    reactcount = pd.read_sql(sql, con=conn)
    return reactcount['count'].values[0]

def count_a(drugid, reactid, con = conn):
    '''
    Figure out the value 'a' (the count of recordings associating 
    with given drug and focusing adverse reaction) 
    in the Disproportional Method
    '''
    sql = (f"SELECT count(*) FROM cdir_fo WHERE drugbank_id = '{drugid}' AND reactid = '{reactid}';")
    reactcount = pd.read_sql(sql, con=conn)
    return reactcount['count'].values[0]

def ror_ci(a, b, c, d, ror):
    ror_upper_ci_power = math.log(ror) + 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    ror_lower_ci_power = math.log(ror) - 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    ror_upper_ci = math.exp(ror_upper_ci_power)
    ror_lower_ci = math.exp(ror_lower_ci_power)
    return '%.2f' % ror_lower_ci, '%.2f' % ror_upper_ci


def prr_ci(a, b, c, d, prr):
    prr_upper_ci_power = math.log(prr) + 1.96 * math.sqrt(1/a + 1/b - 1/(a + c) - 1/(b + d))
    prr_lower_ci_power = math.log(prr) - 1.96 * math.sqrt(1/a + 1/b - 1/(a + c) - 1/(b + d))
    prr_upper_ci = math.exp(prr_upper_ci_power)
    prr_lower_ci = math.sqrt(prr_lower_ci_power)
    return '%.2f' % prr_lower_ci, '%.2f' % prr_upper_ci

In [ ]:
def calculate(con = conn):
    total = 6981059
    sql = ("SELECT DISTINCT drugbank_id FROM drug21_24_map ORDER BY drugbank_id ASC limit 2")
    drugids = pd.read_sql(sql, con = conn)
    for drugid in drugids['drugbank_id']:
        reactids = get_drug_reaction(drugid)
        ac = count_drug(drugid)
        for reactid in reactids:
            ab = count_react(reactid)
            a = count_a(drugid, reactid)
            b = ab - a
            c = ac - a
            d = total - ab - ac + a
            if b == 0 or c == 0:
                a+=0.5
                b+=0.5
                c+=0.5
                d+=0.5
            ror = (a * d) / (b * c)
            ror_lower_ci, ror_upper_ci = ror_ci(a, b, c, d, ror)

            prr = a * (b + d)/(b * ac)
            prr_lower_ci, prr_upper_ci = prr_ci(a, b, c, d, prr)
            cur = conn.cursor()
            cur.execute(f"INSERT INTO FAERSmining (drugid, reactid, a, b, c, d, ror, ror_lower_ci, ror_upper_ci, prr, prr_lower_ci, prr_upper_ci) VALUES('{drugid}', '{reactid}', {a}, {b}, {c}, {d}, {ror}, {ror_lower_ci}, {ror_upper_ci}, {prr}, {prr_lower_ci}, {prr_upper_ci})")
           
